# Le type String

---

## Présentation

- Une String est un autre type alloue, spécialise dans les chaînes de caractères. Contrairement aux Box qui ont une taille fixe, la String est plutôt comme un Vecteur et change de taille selon ce qu'elle contient.

![STRING](pictures/string_proto.png)

In [2]:
let mut s: String = String::new();
s.push_str("Hello");
dbg!(&s);
s.push(' ');
s.push_str("World");
dbg!(&s);
s.clear();
dbg!(&s);

[src/lib.rs:104] &s = "Hello"
[src/lib.rs:107] &s = "Hello World"
[src/lib.rs:109] &s = ""


**Contrairement aux chaînes de caractères des langages bas niveau, une String en Rust contient toujours des caractères UTF8 valides qui peuvent tenir sur n octets (allant de 1 à 4). Ce n'est pas une simple chaîne ASCII.**

## Création de String

### 4 façons utilisées de façon courante

- Il existe plusieurs moyens de créer une chaîne de caractère allouée dynamiquement, en voici une liste non-exhaustive :
> - Utilisation de **String::new()**
> - Les traits **From** et **Into** à partir du type **&str**
> - La méthode **to_string()** du type **&str**
> - Emploi de la **macro format!**

In [9]:
let mut s = String::new(); // String::new()
s.push_str("Hello World !");
dbg!(s);

let s: String = "Ce sont des choses qui arrivent.".into(); // Trait Into
dbg!(s);
let s = String::from("Ce sont des choses qui arrivent."); // Trait From
dbg!(s);
let s = String::from("Ce sont des choses ") + "qui arrivent".into(); // Combo !
dbg!(s);

let s = "Encore un autre moyen...".to_string(); // Methode to_string()
dbg!(s);

let i: usize = 42; // Utilisation de la macro format!
let s = format!("Et voila un dernier moyen, celui ci est plus classe car je peux mettre {} dedans.", 42);
dbg!(s);

[src/lib.rs:106] s = "Hello World !"
[src/lib.rs:109] s = "Ce sont des choses qui arrivent."
[src/lib.rs:111] s = "Ce sont des choses qui arrivent."
[src/lib.rs:113] s = "Ce sont des choses qui arrivent"
[src/lib.rs:116] s = "Encore un autre moyen..."
[src/lib.rs:120] s = "Et voila un dernier moyen, celui ci est plus classe car je peux mettre 42 dedans."


## Les méthodes pour String

> Il existe beaucoup de méthodes pour String, la meilleure chose à faire est de les essayer. Elles sont toutes plus ou moins spécialisées dans la gestion des chaînes de caractère contrairement à ce que serait un Vec\<char\>.

- Il est important de noter aussi que le type String implémente les traits Add & AddAssign ce qui permet d'utiliser l'opérateur *+* :

In [12]:
let mut s = format!("Hello");
s += " ";                         // Utilisation de AddAssign
s += "World!";                    // Utilisation de AddAssign
dbg!(s);

[src/lib.rs:107] s = "Hello World!"


- Par exemple, la méthode replace permet de remplacer une partie de la chaîne de caractères :

In [16]:
let s = "这是一只非常古老的猴子";
let new_string = s.replace("只非常古老的猴子", "头很老的猪");
println!("The new string is {}", new_string);

The new string is 这是一头很老的猪


## Le type &str à partir d'une String

> Tout comme pour les Vector avec les slices, une variable de type &str peut être obtenue à partir d'un type String. Grace a l'opérateur de référencement, on obtient ainsi une &str (ou une slice de type str). Mais cependant, souvenez-vous que le type String peut contenir des éléments de taille différente puisque ce sont des caractères UTF8, et que la notation crochet qui permet de ne récupérer qu'une sous-séquence fonctionne à sur des elements de taille identique.

- Il y a donc des risques de couper des séquences UTF8 en plein milieu d'un caractère :

In [21]:
{
    let s: String = "这是一只非常古老的猴子".into();
    let str = &s;
    dbg!(str);
    let str1 = &s[..3]; // Ca passe
    dbg!(str1);
    let str2 = &s[0..6]; // Ca passe aussi, je connais bien le format UTF8 :)
    dbg!(str2);
    let str3 = &s[1..s.len()]; // Le premier caractere est coupe ici !
    dbg!(str3);
}

[src/lib.rs:25] str = "这是一只非常古老的猴子"
[src/lib.rs:27] str1 = "这"
[src/lib.rs:29] str2 = "这是"
thread '<unnamed>' panicked at 'byte index 1 is not a char boundary; it is inside '这' (bytes 0..3) of `这是一只非常古老的猴子`', src/lib.rs:30:17
stack backtrace:
   0: rust_begin_unwind
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/std/src/panicking.rs:575:5
   1: core::panicking::panic_fmt
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/panicking.rs:64:14
   2: core::str::slice_error_fail_rt
   3: core::str::slice_error_fail
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/str/mod.rs:86:9
   4: <unknown>
   5: <unknown>
   6: evcxr::runtime::Runtime::run_loop
   7: evcxr::runtime::runtime_hook
   8: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


> Ce code n'aurait pas paniqué si ma Sring n'avait contenu que de vulgaires caractères ASCII.

## A propos de From et de Into

**Implémentation du trait From<&str> pour String**
```
impl From<&str> for String
```

**Description du trait From ...**
```
Trait std::convert::From

pub trait From<T>: Sized {
    // Required method
    fn from(value: T) -> Self;
}
```

**... Dans la même description, il est écrit un peu plus loin**
```
Generic Implementations

    From<T> for U implies Into<U> for T
    From is reflexive, which means that From<T> for T is implemented
```

**Et enfin, la description du trait Into.**
```
Trait std::convert::Into

pub trait Into<T>: Sized {
    // Required method
    fn into(self) -> T;
}
```

> Que peut-on dire des traits From et de Into ?

From et Into permettent la conversion entres différents types. Implémenter From\<T\> pour un type en définissant la méthode from() permet aussi d'utiliser la méthode into() pour mon type sans avoir à définir une méthode into().

Enfin, From\<&str\> for String est implémenté ce qui me permet de créer une chaîne de caractère allouée en mémoire à partir de chaînes statiques.